##  Rozwiązywanie układu równań liniowych metodami rozkładu LU  
#### Zadanie 1  
Zaimplementuj metody rozkładu LU:
- Rozkład Crouta  
- Rozkład Doolitla  
- Rozkład Choleskyego  
  
Dla każdej z metod podaj warunki niezbędne aby można ją było zastosować. Sprawdź poprawność działania tych metod.
Przetestuj wydajność algorytmów dla kilku rozmiarów macierzy (podobnie jak w ćwiczeniu 9).

In [58]:
import numpy as np
from scipy import linalg
import time as time
from sklearn.linear_model.tests.test_passive_aggressive import random_state
from sklearn.utils import check_random_state

def croutD(A):
    n=len(A)
    L = np.zeros((n, n))
    U = np.zeros((n, n))

    for k in range(0, n):
        U[k, k] = 1 
        for j in range(k, n):
            sum0 = sum([L[j, s] * U[s, k] for s in range(0, j)])
            L[j, k] = A[j, k] - sum0

        for j in range(k+1, n):
            sum1 = sum([L[k, s] * U[s, j] for s in range(0, k)]) 
            U[k, j] = (A[k, j] - sum1) / L[k, k]
    return L,U

A = np.array([[1,2,3], [2,20,26], [3,26,0]])
croutD(A)

def doolittleD(A):
    n=len(A)
    L = np.zeros((n, n))
    U = np.zeros((n, n))
    
    for i in range(n): 
        for k in range(i, n):  
            sum = 0; 
            for j in range(i): 
                sum += (L[i][j] * U[j][k]); 
            U[i][k] = A[i][k] - sum; 
        for k in range(i, n): 
            if (i == k): 
                L[i][i] = 1;
            else:  
                sum = 0; 
                for j in range(i): 
                    sum += (L[k][j] * U[j][i]); 
                L[k][i] = ((A[k][i] - sum) / U[i][i]);
    return L,U
    
B=np.array([[2,-1,-2],[-4,6,3],[-4,-2,8]])
doolittleD(B)

def choleskyD(A):
    n = len(A)
    L = np.zeros((n, n))
    
    for i in range(n):
        for k in range(i+1):
            tmp_sum = sum(L[i][j] * L[k][j] for j in range(k))
            if (i == k):
                L[i][k] = np.sqrt(A[i][i] - tmp_sum)
            else:
                L[i][k] = (1.0 / L[k][k] * (A[i][k] - tmp_sum))
    return L,L.T 
    
C=[[4,12,-16],[12,37,-43],[-16,-43,98]]
choleskyD(C)
print()

Dla wszystkich metod była sprawdzona poprawność działania(zadanie 2).  
Używamy rozklad (Dane z wykładu) :  
- Doolitla i Crouta :  może być wykorzystany dla dowolnej macierzy
- Choleskiego : jeśli macierz A układu równań jest macierzą rzeczywistą, symetryczną, dodatnio określoną.

In [81]:
def efficiencyTestingCholesky(numberOfMatrixes,matrixDegree):
    start=time.time()
    for i in range(0,numberOfMatrixes):
        generator = check_random_state(random_state)
        A = generator.rand(matrixDegree,matrixDegree)
        U, s, V = linalg.svd(np.dot(A.T, A))
        X = np.dot(np.dot(U, 1.0 + np.diag(generator.rand(matrixDegree))), V)
        X=X*100
        choleskyD(X)
    end=time.time()
    print("Rozklad Choleskiego",end-start,"s")
    
def efficiencyTesting(numberOfMatrixes,matrixDegree):
    start=time.time()
    for i in range(0,numberOfMatrixes):
        X=np.random.randint(-100,100,(matrixDegree,matrixDegree))
        croutD(X)
    end=time.time()
    print("Rozklad Crouta",end-start,"s")
    start=time.time()
    for i in range(0,numberOfMatrixes):
        X=np.random.randint(-100,100,(matrixDegree,matrixDegree))
        doolittleD(X)
    end=time.time()
    print("Rozklad Doolitle`a",end-start,"s")
    

print("Testowanie wydajności dla: ")
print("\t10 macierz 10 stopnia")
efficiencyTestingCholesky(10,10)
efficiencyTesting(10,10)
print("\t50 macierz 50 stopnia")
efficiencyTestingCholesky(50,50)
efficiencyTesting(50,50)
print("\t100 macierz 100 stopnia")
efficiencyTestingCholesky(100,100)
efficiencyTesting(100,100)

Testowanie wydajności dla: 
	10 macierz 10 stopnia
Rozklad Choleskiego 0.0045032501220703125 s
Rozklad Crouta 0.008445024490356445 s
Rozklad Doolitle`a 0.0059833526611328125 s
	50 macierz 50 stopnia
Rozklad Choleskiego 1.1019177436828613 s
Rozklad Crouta 2.074875593185425 s
Rozklad Doolitle`a 2.0264482498168945 s
	100 macierz 100 stopnia
Rozklad Choleskiego 14.96259593963623 s
Rozklad Crouta 31.017005443572998 s
Rozklad Doolitle`a 31.730915546417236 s


#### Zadanie 2
Zapoznaj się z funkcją rozwiązywania układów równań liniowych dostarczoną przez bibliotekę numpy/scipy. Porównaj jej wydajność z własnymi implementacjami.

In [42]:
import numpy as np

def solve(L,U,B):
    n=len(L)
    Y=[None]*n
    Y[0]=B[0]/L[0][0]
    for i in range(1,n):
        s=0
        for j in range(0,i):
            s+=L[i][j]*Y[j]
        Y[i]=(B[i]-s)/L[i][i]
        
    X=[None]*n

    X[n-1]=Y[n-1]/U[n-1][n-1]
    for i in range(n-2,-1,-1):
        s=0
        for j in range(i+1,n):
            s+=U[i][j]*X[j]
        X[i]=(Y[i]-s)/U[i][i]
    return X

def npSolve(A,B):
    return np.linalg.solve(A, B)

A=np.array([[4,12,-16],[12,37,-43],[-16,-43,98]])
B=[4,2,3]

print("Testowania poprawności działania:")
print("\t Wyniki otrzymane przy pomocy numpy:\n",npSolve(A,B))
L,U=doolittleD(A)
print("\t Wyniki otrzymane przy rozkladu Doolitla:\n",solve(L,U,B))
L,U=croutD(A)
print("\t Wyniki otrzymane przy rozkladu Crouta:\n",solve(L,U,B))
L,U=choleskyD(A)
print("\t Wyniki otrzymane przy pomocy rozkładu Choleskyego:\n",solve(L,U,B))

Testowania poprawności działania:
	 Wyniki otrzymane przy pomocy numpy:
 [176.66666667 -48.33333333   7.66666667]
	 Wyniki otrzymane przy rozkladu Doolitla:
 [176.66666666666666, -48.333333333333336, 7.666666666666667]
	 Wyniki otrzymane przy rozkladu Crouta:
 [176.66666666666666, -48.333333333333336, 7.666666666666667]
	 Wyniki otrzymane przy pomocy rozkładu Choleskyego:
 [176.66666666666666, -48.333333333333336, 7.666666666666667]


In [57]:
def generateAB(matrixDegree):
    generator = check_random_state(random_state)
    A = generator.rand(matrixDegree,matrixDegree)
    U, s, V = linalg.svd(np.dot(A.T, A))
    A = np.dot(np.dot(U, 1.0 + np.diag(generator.rand(matrixDegree))), V)
    A=A*100
    X=np.random.randint(-15,15,matrixDegree) 
    B=[]
    for i in range(0,matrixDegree):
        numberToAdd=0
        for j in range(0,matrixDegree):
            numberToAdd+=X[j]*A[i][j]
        B.append(numberToAdd)
    return A,B

def efficiencyTesting(numberOfMatrixes,matrixDegree):
    start=time.time()
    for i in range(0,numberOfMatrixes):
        A,B=generateAB(matrixDegree)
        L,U=choleskyD(A)
        solve(L,U,B)
    end=time.time()
    print("Rozwiązanie przy pomocy rozkladu Choleskiego",end-start,"s")
    
    start=time.time()
    for i in range(0,numberOfMatrixes):
        A,B=generateAB(matrixDegree)
        L,U=croutD(A)
        solve(L,U,B)
    end=time.time()
    print("Rozwiązanie przy pomocy rozkladu Crouta",end-start,"s")
    
    start=time.time()
    for i in range(0,numberOfMatrixes):
        A,B=generateAB(matrixDegree)
        L,U=doolittleD(A)
        solve(L,U,B)
    end=time.time()
    print("Rozwiązanie przy pomocy rozkladu Doolitla",end-start,"s")
    
    start=time.time()
    for i in range(0,numberOfMatrixes):
        A,B=generateAB(matrixDegree)
        npSolve(A,B)
    end=time.time()
    print("Rozwiązanie przy pomocy biblioteki numpy",end-start,"s")
    
    
print("Testowanie wydajności dla: ")
print("\t10 macierz 10 stopnia")
efficiencyTesting(10,10)
print("\t50 macierz 50 stopnia")
efficiencyTesting(50,50)
print("\t100 macierz 100 stopnia")
efficiencyTesting(100,100)

Testowanie wydajności dla: 
	10 macierz 10 stopnia
Rozwiązanie przy pomocy rozkladu Choleskiego 0.007978677749633789 s
Rozwiązanie przy pomocy rozkladu Crouta 0.00997304916381836 s
Rozwiązanie przy pomocy rozkladu Doolitla 0.008980035781860352 s
Rozwiązanie przy pomocy biblioteki numpy 0.0049855709075927734 s
	50 macierz 50 stopnia
Rozwiązanie przy pomocy rozkladu Choleskiego 1.5416815280914307 s
Rozwiązanie przy pomocy rozkladu Crouta 2.379384756088257 s
Rozwiązanie przy pomocy rozkladu Doolitla 2.2950003147125244 s
Rozwiązanie przy pomocy biblioteki numpy 0.503514289855957 s
	100 macierz 100 stopnia
Rozwiązanie przy pomocy rozkladu Choleskiego 18.903281688690186 s
Rozwiązanie przy pomocy rozkladu Crouta 36.01801252365112 s
Rozwiązanie przy pomocy rozkladu Doolitla 34.06443238258362 s
Rozwiązanie przy pomocy biblioteki numpy 3.7017428874969482 s


### Wniosek:  
Możemy zrobić wniosek, że najsprytniejszą jest metoda zaimplementowana w bibliotece numpy, na drugim miejscu metoda, która wykorzystuje rozkład Choleskiego, ale nie działa ona dla wszystkich układów. Metody wykorzystujące rozkład Crouta i Doolitla działają prawie w tym samym czasie.